# Imputer - Nettoyage des données

* Dans pandas il y a df.fillna()
* Dans le module **impute** de sklearn
* Ces transformers permettent de nettoyer le dataset en éliminant, remplacant des valeurs manquantes

On va voir : 
* SimpleImputer
* KNNImputer
* MissingIndicator


## SimpleImputer()




In [15]:
import numpy as np
from sklearn.impute import SimpleImputer

X_train = np.array([[10,3],
              [0,4],
              [5,3],
              [np.nan, 3]
              ])
# strategy = mean, median, most_frequent, constant
imputer = SimpleImputer(missing_values = np.nan, 
                        strategy = "mean")
imputer.fit_transform(X_train)

array([[10.,  3.],
       [ 0.,  4.],
       [ 5.,  3.],
       [ 5.,  3.]])

* La moyenne de la 1ere colonne remplace bien np.nan
* On peut ensuite appliquer ce Transformer à d'autres dataset
* X_test par exemple

Ci-dessous bien voir qu'on utilise transform() et pas fit_transform() car on veut appliquer les moyennes du Train set au Test set. 

De cette façon on est sûr de traiter toutes les données de la même façon qu'on a traité les données du Train set.

In [16]:
X_test = np.array([[12, 5],
              [40, 2],
              [5, 5],
              [np.nan, np.nan]
              ])
imputer.transform(X_test)          # transform()


array([[12.  ,  5.  ],
       [40.  ,  2.  ],
       [ 5.  ,  5.  ],
       [ 5.  ,  3.25]])

Pourquoi on calcule pas les moyennes sur l'ensemble Train set + Test set et qu'on remplace les valeurs manquantes par ces moyennes?

Si on fait ça il y a une "fuite" d'information du Test set vers le Train set. Certaines information issues du Test set sont utilisées pour entrainer le modèle. C'est pas du tout une bonne idée.

## KNNImputer()

* Depuis ver 0.22
* Remplace les valeurs manquantes par celles des plus proches voisins

![alt](assets/KNNImputer.png)

In [18]:
import numpy as np
from sklearn.impute import KNNImputer

X = np.array([[1, 100],
              [200, 21],
              [3, 15],
              [np.nan, 20]])
imputer = KNNImputer(n_neighbors=1)
imputer.fit_transform(X)

array([[  1., 100.],
       [200.,  21.],
       [  3.,  15.],
       [200.,  20.]])

* Penser à utiliser GridSearchCV() afin de trouver le nombre optimal de voisins à passer à KNNIputer().
* Voir plus bas

## MissingIndicator()
* Valeur booleene qui indique une valeur absente
  

In [24]:
import numpy as np
from sklearn.impute import MissingIndicator

X = np.array([[1, 100],
              [2, 30],
              [3, 15],
              [np.nan, np.nan]])
Indiq = MissingIndicator().fit_transform(X)
print(Indiq)
print(Indiq.sum()) # on peut déterminer 
print(f"Il y a {100*Indiq.sum()/Indiq.size}% de données manquantes dans le dataset")


[[False False]
 [False False]
 [False False]
 [ True  True]]
2
Il y a 25.0% de données manquantes dans le dataset


* On considère ici que les 2 dernières valeurs (classe, deck par exemple) sont celles d'un matelo. 
* Il n'a pas de class ni de pont
* On va créer une nouvelle colonne pour distinguer les matelos

In [25]:
import numpy as np
from sklearn.impute import MissingIndicator
from sklearn.pipeline import make_union

X = np.array([[1, 100],
              [2, 30],
              [3, 15],
              [np.nan, np.nan]])
MissingIndicator().fit_transform(X)
pipeline = make_union (SimpleImputer(strategy="constant", fill_value=-99), 
                       MissingIndicator())
pipeline.fit_transform(X)


array([[  1., 100.,   0.,   0.],
       [  2.,  30.,   0.,   0.],
       [  3.,  15.,   0.,   0.],
       [-99., -99.,   1.,   1.]])

* On retrouve les colonnes qui dans lesqeulles on a fait passer le SimpleIputer
* make_union a aussi créé 2 colonnes qui permettent de savoir si c'est un passager ou un membre d'équipage

### À noter
On se sert du manque d'information (np.nan) pour créer une nouvelle information pour entrainer le modèle

## Application
Utiliser GridSearchCV pour trouver les meilleurs paramètres du pipeline

In [31]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

titanic = sns.load_dataset("titanic")
#print(titanic.shape)

y = titanic["survived"]
X = titanic[["pclass", "age"]]

# random_state permet d'avoir toujours les 2 mêmes jeux
# test_size permet de faire 80-20%
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=.2, random_state=0 )

# SGDClassifier = Linear classifiers (SVM, logistic regression, etc.) with SGD training.
model = make_pipeline(KNNImputer(), SGDClassifier())
model.get_params()



{'memory': None,
 'steps': [('knnimputer', KNNImputer()), ('sgdclassifier', SGDClassifier())],
 'verbose': False,
 'knnimputer': KNNImputer(),
 'sgdclassifier': SGDClassifier(),
 'knnimputer__add_indicator': False,
 'knnimputer__copy': True,
 'knnimputer__keep_empty_features': False,
 'knnimputer__metric': 'nan_euclidean',
 'knnimputer__missing_values': nan,
 'knnimputer__n_neighbors': 5,
 'knnimputer__weights': 'uniform',
 'sgdclassifier__alpha': 0.0001,
 'sgdclassifier__average': False,
 'sgdclassifier__class_weight': None,
 'sgdclassifier__early_stopping': False,
 'sgdclassifier__epsilon': 0.1,
 'sgdclassifier__eta0': 0.0,
 'sgdclassifier__fit_intercept': True,
 'sgdclassifier__l1_ratio': 0.15,
 'sgdclassifier__learning_rate': 'optimal',
 'sgdclassifier__loss': 'hinge',
 'sgdclassifier__max_iter': 1000,
 'sgdclassifier__n_iter_no_change': 5,
 'sgdclassifier__n_jobs': None,
 'sgdclassifier__penalty': 'l2',
 'sgdclassifier__power_t': 0.5,
 'sgdclassifier__random_state': None,
 'sgdcla

Par rapport à une fonction pandas pour faire de l'imputation, KNNImputer permet d'utiliser le Transformer avec GridSearchCV() afin d'en optimiser les caractéristiques

Utiliser la sortie de model.get_params() pour trouver les noms des paramètres 

In [34]:
from sklearn.model_selection import GridSearchCV

params = {
  "knnimputer__n_neighbors" : [1, 2, 3, 4]
  # on pourrait ajouter un paramètre de sgdclassifier__xxx à optimiser
}

grid = GridSearchCV (model, param_grid=params, cv=5)

# on entraine la grid
grid.fit(X_train, y_train) 

# affiche le nombre optimal de voisins à utiliser
grid.best_params_

{'knnimputer__n_neighbors': 3}

Donc ici, le nombre optimal de voisins à utiliser c'est 3

In [37]:
imputer = KNNImputer(n_neighbors=3)
imputer.fit_transform(X_train)


array([[ 3., 21.],
       [ 2., 31.],
       [ 2., 31.],
       ...,
       [ 3., 21.],
       [ 3., 36.],
       [ 2., 60.]])

In [51]:
model = make_pipeline(KNNImputer(n_neighbors=3), SGDClassifier())
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.7039106145251397